# Создание и заполнение данных БД Postgre

In [9]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor


In [10]:
POSTGRESQL_HOST = '10.129.0.25'

In [11]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


In [12]:
!psql -c 'create database test;'

ERROR:  database "test" already exists


In [13]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [14]:
%%sql postgresql:///test
    SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

0 rows affected.


table


## Примеры SQL запросов 
1. Представим запрос для создания таблицы users c двумя полями:
```
%%sql postgresql:///test
    CREATE TABLE users (
        login varchar(10) NOT NULL, 
        password varchar(100)
    );
```

2. Чтобы добавить запись в таблицу, небходимо выполнить следующий запрос:

```
%%sql postgresql:///test
   INSERT INTO users VALUES ('monit', 'querty')
```

3. Получить данные из таблицы:
```
%%sql postgresql:///test
   SELECT * FROM users
```

4. Удалить таблицу:
```
!psql -d test -c 'DROP TABLE users'
```

Считаем данные из файла datasets/aggrigation_logs_per_week.csv.

Выведем первую строку с заголовками. 

In [24]:
cd ~

/home/jovyan


In [25]:
import csv

with open('datasets/aggrigation_logs_per_week.csv') as file:
    reader = csv.reader(file)
    headers = next(reader)
print (headers)

['courseid', 'userid', 'num_week', 's_all', 's_all_avg', 's_course_viewed', 's_course_viewed_avg', 's_q_attempt_viewed', 's_q_attempt_viewed_avg', 's_a_course_module_viewed', 's_a_course_module_viewed_avg', 's_a_submission_status_viewed', 's_a_submission_status_viewed_avg', 'NameR_Level', 'Name_vAtt', 'Depart', 'Name_OsnO', 'Name_FormOPril', 'LevelEd', 'Num_Sem', 'Kurs', 'Date_vAtt']


Вам предоставлен набор данных с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу.
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Создадим таблицу с логами студентов в тестовой БД

In [26]:
!psql -d test -c 'CREATE TABLE logs ( \
    courseid varchar(5), \
    userid varchar(5), \
    num_week smallint, \
    s_all smallint, \
    s_all_avg varchar(10), \
    s_course_viewed smallint, \
    s_course_viewed_avg varchar(10), \
    s_q_attempt_viewed smallint, \
    s_q_attempt_viewed_avg varchar(10), \
    s_a_course_module_viewed smallint, \
    s_a_course_module_viewed_avg varchar(10), \
    s_a_submission_status_viewed smallint, \
    s_a_submission_status_viewed_avg varchar(10), \
    NameR_Level varchar(20), \
    Name_vAtt varchar(20), \
    Depart varchar(5), \
    Name_OsnO smallint, \
    Name_FormOPril smallint, \
    LevelEd smallint, \
    Num_Sem smallint, \
    Kurs smallint, \
    Date_vAtt varchar(20)\
);'

CREATE TABLE


Добавим данные из файла aggrigation_logs_per_week.csv в созданную таблицу.


In [27]:
!psql -d test \
    -c "\copy logs FROM 'datasets/aggrigation_logs_per_week.csv' DELIMITER ',' CSV HEADER;"

COPY 414528


Выведем первые 10 записей.

In [35]:
%%sql postgresql:///test
    SELECT * FROM logs LIMIT 10

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71710,28778,10,0,3.2,0,1,0,0,0,0,0,0,4,Экзамен,40,1,1,1,4,3,25.06.2022
71710,28805,6,0,0,0,0,0,0,0,0,0,0,3,Экзамен,40,2,1,1,4,3,25.06.2022
71717,24399,6,6,6,2,2,0,0,2,2,2,2,5,Экзамен,40,1,1,1,6,4,27.06.2022
71732,25038,29,0,0.125,0,0,0,0,0,0,0,0,2,Экзамен,40,1,2,1,4,3,29.06.2022
71733,23014,6,0,0,0,0,0,0,0,0,0,0,3,Экзамен,40,1,2,1,8,5,17.06.2022
71733,24518,7,0,0,0,0,0,0,0,0,0,0,4,Экзамен,40,1,1,1,6,4,18.06.2022
71758,31265,6,17,17,8,8,0,0,0,0,0,0,2,Экзамен,35,1,1,1,4,3,18.06.2022
71792,32027,19,0,0.2857,0,0.2143,0,0,0,0,0,0,3,Экзамен,30,1,1,1,4,3,21.06.2022
71797,34739,15,67,34.5,9,5.7,0,0,13,9,11,6.8,4,Экзамен,30,1,1,1,2,2,02.07.2022
71852,33789,6,0,0,0,0,0,0,0,0,0,0,4,Экзамен,3,1,1,1,2,2,04.07.2022


## Задание 1:

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

In [29]:
%%sql postgresql:///test
UPDATE logs
SET 
    s_all_avg = REPLACE(s_all_avg, ',', '.'),
    s_course_viewed_avg = REPLACE(s_course_viewed_avg, ',', '.'),
    s_q_attempt_viewed_avg = REPLACE(s_q_attempt_viewed_avg, ',', '.'),
    s_a_course_module_viewed_avg = REPLACE(s_a_course_module_viewed_avg, ',', '.'),
    s_a_submission_status_viewed_avg = REPLACE(s_a_submission_status_viewed_avg, ',', '.');

414528 rows affected.


[]

In [33]:
%%sql postgresql:///test
SELECT * FROM logs LIMIT 10

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71596,29127,6,0,0,0,0,0,0,0,0,0,0,4,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29213,15,3,11.3,1,2.6,0,0,1,3.6,1,2.6,5,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29235,29,0,22.125,0,3.75,0,8.75,0,3,0,2.3333,5,Экзамен,6,1,1,1,4,3,27.06.2022
71596,29243,6,31,31,8,8,0,0,7,7,5,5,5,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29263,6,2,2,2,2,0,0,0,0,0,0,4,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29426,13,0,9.5,0,2,0,4,0,1.125,0,1.125,4,Экзамен,6,1,1,1,4,3,17.06.2022
71596,36513,9,0,0.5,0,0.5,0,0,0,0,0,0,3,Экзамен,6,1,1,1,4,3,24.06.2022
71615,25360,6,0,0,0,0,0,0,0,0,0,0,3,Экзамен,6,1,1,1,4,3,02.07.2022
71615,29139,13,5,62,3,18.875,0,0,1,16,1,14,5,Экзамен,6,1,1,1,4,3,02.07.2022
71635,35394,6,3,3,1,1,0,0,0,0,0,0,5,Экзамен,6,1,1,1,2,2,17.06.2022


## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





In [36]:
%%sql postgresql:///test
SELECT COUNT(DISTINCT depart) AS department_count
FROM logs;

1 rows affected.


department_count
43


##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 

У какой кафедры больше всего курсов на портале?

In [37]:
%%sql postgresql:///test
SELECT 
    depart,
    COUNT(DISTINCT courseid) AS course_count
FROM logs
GROUP BY depart
ORDER BY course_count DESC;

43 rows affected.


depart,course_count
11,53
19,42
23,42
8,40
15,36
4,35
9,35
21,33
14,33
3,33


In [38]:
%%sql postgresql:///test
SELECT 
    depart,
    COUNT(DISTINCT courseid) AS course_count
FROM logs
GROUP BY depart
ORDER BY course_count DESC
LIMIT 1;

1 rows affected.


depart,course_count
11,53


## Задание 4:

Имеется информация о кодировании каждой кафедры:

| Id  | Name     | Id  | Name     | Id  | Name     | Id  | Name     |
|-----|---------|-----|---------|-----|---------|-----|---------|
| 1   | АиИИ    | 12  | ИиИБ    | 23  | ПОиД    | 34  | УСиБА    |
| 2   | АСУ     | 13  | ИТМ     | 24  | Психол. | 35  | Физики   |
| 3   | АЭПиМ   | 14  | ЛиП     | 25  | ПЭиБЖД  | 36  | Физкульт.|
| 4   | БИиИТ   | 15  | ЛиУТС   | 26  | РМПИ    | 37  | Химии    |
| 5   | ВИ      | 16  | ЛПиМ    | 27  | РЯОЯиМК | 38  | ХОМ      |
| 6   | ВТиП    | 17  | Менеджм.| 28  | СРиППО  | 39  | ЦДОМ     |
| 7   | ГМДиОПИ | 18  | МиТОДиМ | 29  | CC      | 40  | ЭиМЭ     |
| 8   | ГМиТТК  | 19  | МиХТ    | 30  | ТиЭС    | 41  | Эконом.  |
| 9   | ГМУиУП  | 20  | ПиСЗ    | 31  | ТОМ     | 42  | ЭПП      |
| 10  | Дизайна | 21  | ПиЭММО  | 32  | ТССА    | 43  | ЯиЛ      |
| 11  | ДиСО    | 22  | ПМиИ    | 33  | УиИС    |     |          |


Создайте отдельно таблицу со справочником (номер кафедры и название(аббревиатура) кафедры) departments.

Внесите данные в таблицу. 

In [39]:
%%sql postgresql:///test
CREATE TABLE departments (
    id smallint PRIMARY KEY,
    name varchar(20) NOT NULL
);

Done.


[]

In [40]:
%%sql postgresql:///test
INSERT INTO departments (id, name) VALUES
(1, 'АиИИ'), (2, 'АСУ'), (3, 'АЭПиМ'), (4, 'БИиИТ'), (5, 'ВИ'),
(6, 'ВТиП'), (7, 'ГМДиОПИ'), (8, 'ГМиТТК'), (9, 'ГМУиУП'), (10, 'Дизайна'),
(11, 'ДиСО'), (12, 'ИиИБ'), (13, 'ИТМ'), (14, 'ЛиП'), (15, 'ЛиУТС'),
(16, 'ЛПиМ'), (17, 'Менеджм.'), (18, 'МиТОДиМ'), (19, 'МиХТ'), (20, 'ПиСЗ'),
(21, 'ПиЭММО'), (22, 'ПМиИ'), (23, 'ПОиД'), (24, 'Психол.'), (25, 'ПЭиБЖД'),
(26, 'РМПИ'), (27, 'РЯОЯиМК'), (28, 'СРиППО'), (29, 'CC'), (30, 'ТиЭС'),
(31, 'ТОМ'), (32, 'ТССА'), (33, 'УиИС'), (34, 'УСиБА'), (35, 'Физики'),
(36, 'Физкульт.'), (37, 'Химии'), (38, 'ХОМ'), (39, 'ЦДОМ'), (40, 'ЭиМЭ'),
(41, 'Эконом.'), (42, 'ЭПП'), (43, 'ЯиЛ');

43 rows affected.


[]

## Задание 5:

Выполните задание 3, но теперь нужно вывести не код кафедры, а ее название и количество курсов, которые за ней закреплены. 

У какой кафедры больше всего курсов?

In [41]:
%%sql postgresql:///test
SELECT 
    d.name AS department_name,
    COUNT(DISTINCT l.courseid) AS course_count
FROM logs l
JOIN departments d ON l.depart::smallint = d.id
GROUP BY d.name
ORDER BY course_count DESC;

43 rows affected.


department_name,course_count
ДиСО,53
МиХТ,42
ПОиД,42
ГМиТТК,40
ЛиУТС,36
БИиИТ,35
ГМУиУП,35
АЭПиМ,33
ЛиП,33
ПиЭММО,33


In [42]:
%%sql postgresql:///test
SELECT 
    d.name AS department_name,
    COUNT(DISTINCT l.courseid) AS course_count
FROM logs l
JOIN departments d ON l.depart::smallint = d.id
GROUP BY d.name
ORDER BY course_count DESC
LIMIT 1;

1 rows affected.


department_name,course_count
ДиСО,53


## Задание 6:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




In [43]:
%%sql postgresql:///test
WITH course_departs AS (
    SELECT 
        courseid,
        COUNT(DISTINCT depart) AS depart_count
    FROM logs
    GROUP BY courseid
    HAVING COUNT(DISTINCT depart) > 1
)
SELECT 
    cd.courseid,
    COUNT(*) AS multi_depart_courses,
    STRING_AGG(d.name, ', ') AS department_names
FROM course_departs cd
JOIN logs l ON cd.courseid = l.courseid
JOIN departments d ON l.depart::smallint = d.id
GROUP BY cd.courseid;

60 rows affected.


[('71495', 1608, 'ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ... (9396 characters truncated) ...  ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ'),
 ('71508', 1176, 'ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ... (6756 characters truncated) ...  ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ'),
 ('71541', 1536, 'ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ... (8916 characters truncated) ...  ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ'),
 ('71547', 1080, 'УиИС, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ... (6180 characters truncated) ...  УиИС, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ'),
 ('71549', 96, 'ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТК, ГМиТТ ... (324 characters truncated) ...  ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС, ТиЭС'),
 ('71571', 1680, 'ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ... (9828 characters truncated) ...  ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ, ПиСЗ'),
 ('71632', 264, 'ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, CC, C ... (1236 characters truncated) ...  ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП, ВТиП'),
 ('71736', 720, 'ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиОПИ, ГМДиО ... (4092 characters truncated) ...  ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ, ЭиМЭ'),
 ('71852', 1392, 'АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭ ... (8052 characters truncated) ... иМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ'),
 ('71857', 1344, 'АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭ ... (7716 characters truncated) ... ЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, ЭПП, АЭПиМ'),
 ('71884', 840, 'АЭПиМ, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭП ... (4668 characters truncated) ... иМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ, АЭПиМ'),
 ('71892', 2616, 'ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, ЭПП, Э

In [44]:
%%sql postgresql:///test
SELECT COUNT(*) AS multi_depart_course_count
FROM (
    SELECT courseid
    FROM logs
    GROUP BY courseid
    HAVING COUNT(DISTINCT depart) > 1
) AS multi_depart_courses;

1 rows affected.


multi_depart_course_count
60


## Задание 7:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


In [45]:
%%sql postgresql:///test
SELECT 
    namer_level,
    COUNT(DISTINCT userid) AS count
FROM logs
GROUP BY namer_level
ORDER BY namer_level;

4 rows affected.


namer_level,count
2,1069
3,1884
4,3243
5,3407


## Задание 8:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за весь период обучения).

In [46]:
%%sql postgresql:///test
SELECT 
    userid,
    COUNT(*) AS log_count
FROM logs
GROUP BY userid
ORDER BY log_count DESC
LIMIT 1;

1 rows affected.


userid,log_count
35250,120


## Задание 9:

Выведите по каждой недели среднее количество всех событий на портале.

In [47]:
%%sql postgresql:///test
SELECT 
    num_week,
    ROUND(AVG(s_all), 2) AS avg_events
FROM logs
GROUP BY num_week
ORDER BY num_week;

24 rows affected.


num_week,avg_events
6,13.80
7,9.62
8,8.03
9,9.39
10,8.21
11,10.02
12,9.38
13,10.01
14,9.86
15,10.35


## Задание 10: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

In [50]:
%%sql postgresql:///test
SELECT 
    d.name AS department_name,
    COUNT(DISTINCT l.userid) AS excellent_students
FROM logs l
JOIN departments d ON l.depart::smallint = d.id
WHERE l.namer_level = '5'
GROUP BY d.name
ORDER BY excellent_students DESC
LIMIT 1;

1 rows affected.


department_name,excellent_students
ДиСО,310


In [49]:
%%sql postgresql:///test
SELECT 
    d.name AS department_name,
    COUNT(DISTINCT l.userid) AS poor_students
FROM logs l
JOIN departments d ON l.depart::smallint = d.id
WHERE l.namer_level = '2'
GROUP BY d.name
ORDER BY poor_students DESC
LIMIT 1;

1 rows affected.


department_name,poor_students
Эконом.,72
